In [127]:
# extension that autoformats all code to PEP8
%load_ext nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [128]:
import numpy as np
import pandas as pd
import plotly.express as px
import statsmodels.api as sm
from sklearn.linear_model import ElasticNet
from IPython.display import display
import plotly.express as px

<IPython.core.display.Javascript object>

In [129]:
# Using updated dataframe from preprocessing notebook
df = pd.read_csv("../data/listings.csv")
display(df)

,Unnamed: 0,name,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,...,neighbourhood_Williamsburg,neighbourhood_Willowbrook,neighbourhood_Windsor Terrace,neighbourhood_Woodhaven,neighbourhood_Woodlawn,neighbourhood_Woodside,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,days_since_review
0,0,Clean & quiet apt home by the park,2787,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,...,0,0,0,0,0,0,0,1,0,-1234
1,1,Skylit Midtown Castle,2845,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,...,0,0,0,0,0,0,1,0,0,-1020
2,3,Cozy Entire Floor of Brownstone,4869,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,...,0,0,0,0,0,0,1,0,0,-975
3,4,Entire Apt: Spacious Studio/Loft by central park,7192,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,...,0,0,0,0,0,0,1,0,0,-1203
4,5,Large Cozy 1 BR Apartment In Midtown East,7322,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,...,0,0,0,0,0,0,1,0,0,-988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38832,48782,Lovely Privet Bedroom with Privet Restroom,83554966,Manhattan,Upper East Side,40.78099,-73.95366,Private room,129,1,...,0,0,0,0,0,0,0,1,0,-973
38833,48790,No.2 with queen size bed,257683179,Queens,Flushing,40.75104,-73.81459,Private room,45,1,...,0,0,0,0,0,0,0,1,0,-973
38834,48799,Seas The Moment,211644523,Staten Island,Great Kills,40.54179,-74.14275,Private room,235,1,...,0,0,0,0,0,0,0,1,0,-973
38835,48805,1B-1B apartment near by Metro,273841667,Bronx,Mott Haven,40.80787,-73.92400,Entire home/apt,100,1,...,0,0,0,0,0,0,1,0,0,-973


<IPython.core.display.Javascript object>

$$
1 / (2 * n_{samples}) * ||y - Xw||^2_2
+ alpha * l_1 ratio * ||w||_1
+ 0.5 * alpha * (1 - l_1 ratio) * ||w||^2_2
$$ 
or 
$$
a * ||w||_1 + 0.5 * b * ||w||_2^2
$$

In [130]:
# y = df["price"]
df = df.drop(
    [
        "Unnamed: 0",
        "name",
        "host_id",
        "neighbourhood_group",
        "neighbourhood",
        "latitude",
        "longitude",
        "room_type",
        "last_review",
    ],
    axis=1,
)
display(df)
display(y)

,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,neighbourhood_group_Queens,...,neighbourhood_Williamsburg,neighbourhood_Willowbrook,neighbourhood_Windsor Terrace,neighbourhood_Woodhaven,neighbourhood_Woodlawn,neighbourhood_Woodside,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,days_since_review
0,149,1,9,0.21,6,365,0,1,0,0,...,0,0,0,0,0,0,0,1,0,-1234
1,225,1,45,0.38,2,355,0,0,1,0,...,0,0,0,0,0,0,1,0,0,-1020
2,89,1,270,4.64,1,194,0,1,0,0,...,0,0,0,0,0,0,1,0,0,-975
3,80,10,9,0.10,1,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,-1203
4,200,3,74,0.59,1,129,0,0,1,0,...,0,0,0,0,0,0,1,0,0,-988
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38832,129,1,1,1.00,1,147,0,0,1,0,...,0,0,0,0,0,0,0,1,0,-973
38833,45,1,1,1.00,6,339,0,0,0,1,...,0,0,0,0,0,0,0,1,0,-973
38834,235,1,1,1.00,1,87,0,0,0,0,...,0,0,0,0,0,0,0,1,0,-973
38835,100,1,2,2.00,1,40,1,0,0,0,...,0,0,0,0,0,0,1,0,0,-973


0        149
1        225
2         89
3         80
4        200
        ... 
38832    129
38833     45
38834    235
38835    100
38836     30
Name: price, Length: 38837, dtype: int64

<IPython.core.display.Javascript object>

In [131]:
# Create training data, holding out 10% of data for testing
from sklearn.model_selection import train_test_split  # Returns pandas dataframe

Xtrain, Xtest, Ytrain, Ytest = train_test_split(
    df.drop("price", axis=1), df["price"], test_size=0.2, random_state=42
)
# Ytrain = Xtrain["price"]
# Xtrain, Xtest = Xtrain.drop("price", axis=1), Xtest.drop("price", axis=1)

display(Xtrain)
display(Xtest)
display(Ytrain)

,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,neighbourhood_group_Queens,neighbourhood_group_Staten Island,...,neighbourhood_Williamsburg,neighbourhood_Willowbrook,neighbourhood_Windsor Terrace,neighbourhood_Woodhaven,neighbourhood_Woodlawn,neighbourhood_Woodside,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,days_since_review
6263,3,153,2.90,1,238,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,-985
18571,2,76,2.78,2,270,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,-986
31589,3,2,0.38,1,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,-1014
25228,1,35,2.18,1,356,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,-980
32717,1,28,4.57,1,111,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,-978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,4,119,2.23,1,273,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,-990
11284,3,7,0.21,13,74,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,-1034
38158,1,5,5.00,1,124,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,-984
860,3,25,0.29,1,0,0,1,0,0,0,...,1,0,0,0,0,0,1,0,0,-1170


,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,neighbourhood_group_Bronx,neighbourhood_group_Brooklyn,neighbourhood_group_Manhattan,neighbourhood_group_Queens,neighbourhood_group_Staten Island,...,neighbourhood_Williamsburg,neighbourhood_Willowbrook,neighbourhood_Windsor Terrace,neighbourhood_Woodhaven,neighbourhood_Woodlawn,neighbourhood_Woodside,room_type_Entire home/apt,room_type_Private room,room_type_Shared room,days_since_review
8022,2,18,0.37,1,280,0,1,0,0,0,...,1,0,0,0,0,0,0,1,0,-1021
3875,6,1,0.02,1,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,-2869
18829,1,15,0.62,1,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,-1407
32406,2,13,1.83,2,97,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,-992
26597,30,2,0.25,6,255,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,-1177
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2923,5,80,1.16,2,298,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,-1002
14280,2,44,1.33,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,-980
11098,1,1,0.02,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,-2262
17025,1,4,0.14,1,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,-1816


6263       80
18571     120
31589    1115
25228     120
32717      31
         ... 
6265      150
11284     189
38158     275
860       160
15795      95
Name: price, Length: 31069, dtype: int64

<IPython.core.display.Javascript object>

In [132]:
enm = ElasticNet(alpha=0.01, l1_ratio=0.99)
enm.fit(Xtrain, Ytrain)

C:\Users\Gianni\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 25822908.897940516, tolerance: 127893.7364905854
  model = cd_fast.enet_coordinate_descent(


ElasticNet(alpha=0.01, l1_ratio=0.99)

<IPython.core.display.Javascript object>

In [133]:
# Lets take the coefficients of training and the names of our columns and put them into a dataframe
variables = Xtrain.columns.tolist()
model_coef = pd.DataFrame({"coef": enm.coef_, "names": variables})


# show all the coef that are non zero in descending order
display(model_coef[model_coef["coef"] != 0].sort_values(by="coef", ascending=False))


,coef,names
205,236.034463,neighbourhood_Tribeca
182,146.957077,neighbourhood_Sea Gate
228,95.168046,room_type_Entire home/apt
7,77.174510,neighbourhood_group_Manhattan
85,72.289916,neighbourhood_Flatiron District
...,...,...
130,-35.514223,neighbourhood_Marble Hill
103,-43.293930,neighbourhood_Harlem
139,-45.438698,neighbourhood_Morningside Heights
214,-67.941326,neighbourhood_Washington Heights


<IPython.core.display.Javascript object>

In [134]:
# Now lets predict with our testing data
predictions = enm.predict(Xtest)

# Compared to the actual values
pred_v_act = pd.DataFrame({"predictions": enm.predict(Xtest), "actual": Ytest})
display(pred_v_act)

from sklearn.metrics import *

score = r2_score(Ytest, predictions)
print("The r-squared value is ", score)
print("The adjusted r-squared value is ", 1 - (((1 - score) * (len(pred_v_act.index) - 1))/(len(pred_v_act) - len(pred_v_act.columns) - 1)))


,predictions,actual
8022,126.275218,90
3875,196.816786,150
18829,177.008573,145
32406,131.895802,120
26597,181.033413,62
...,...,...
2923,76.235340,56
14280,28.570601,60
11098,124.940736,60
17025,213.886243,97


The r-squared value is  0.20657407941021844
The adjusted r-squared value is  0.20636971986853392


<IPython.core.display.Javascript object>